# Implementing C3D Model for Video Classification

In [1]:
%load_ext autoreload
%autoreload 2

from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import sys
import os
import time

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

 
# adding video-download folder to the system path
sys.path.insert(0, '/workspace/youtube-humpback-whale-classifier/video-download')
 
# importing read_frames_hdf5 function
from hdf5_data_loading import read_frames_hdf5

#ngc workspace path (where we keep our data)
workspace_path = '/mount/data'

In [ ]:
import wandb

#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

wandb.run.name = "resnet-data-distribution"

In [2]:
print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU'))) #1 if we select GPU mode in Colab Notebook, 0 if running on local machine

Num GPUs available:  4


## Load Dataset

In [2]:
#load dataset in
data = pd.read_csv(workspace_path + '/downloaded_videos.csv')
y = data.pop('relevant')
X = data

## Load Frames

In [4]:
#load in frames for all videos
start = time.time()

N = X.shape[0] #number of videos in our dataset
videos = np.empty((N, 30, 224, 224, 3), dtype=np.uint8)
labels = np.empty(N, dtype = np.uint8)

for i, video in enumerate(list(X.renamed_title)):
    if i % 50 == 0:
        print(f'Loading frames for video {i}...')
        
    clip_name = video.replace("_", "_clip_").replace(".mp4", "")
    frames, frame_labels = read_frames_hdf5(clip_name) #returns frames array of shape (461, 224, 224, 3)
    
    videos[i, ...] = frames[15:45] #shortened videos bc of memory issue - each video is reduced to 30 frames
    labels[i] = frame_labels[0] #all frames have the same label since label is assigned to overall video

stop = time.time()
print(f'Done loading frames in {stop-start} seconds.')
videos.shape

Loading frames for video 0...
Loading frames for video 50...
Loading frames for video 100...
Loading frames for video 150...
Loading frames for video 200...
Loading frames for video 250...
Loading frames for video 300...
Loading frames for video 350...
Done loading frames in 78.97042894363403 seconds.


(364, 30, 224, 224, 3)

In [27]:
# # videos[0][15:45].shape

# short_videos = np.empty((364, 30, 224, 224, 3), dtype=np.uint8)
# for i in range(364):
#     short_videos[i, ...] = videos[i][15:45]
# short_videos.shape

(364, 30, 224, 224, 3)

## Split Data

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

train_index = list(X_train.index)
test_index = list(X_test.index)

# index data accordingly
train_videos, train_labels = videos[train_index], labels[train_index]
test_videos, test_labels = videos[test_index], labels[test_index]

# reshape label arrays as horizontal arrays
train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

In [6]:
print(train_videos.shape)
print(test_videos.shape)

(291, 30, 224, 224, 3)
(73, 30, 224, 224, 3)


## Converting Data into TF Format

In [7]:
train_tf = tf.data.Dataset.from_tensor_slices((train_videos, train_labels))

2022-07-25 02:54:13.794101: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-25 02:54:17.143646: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14649 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-07-25 02:54:17.152894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14649 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:86:00.0, compute capability: 7.0
2022-07-25 02:54:17.154830: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/repli

In [8]:
train_tf

<TensorSliceDataset element_spec=(TensorSpec(shape=(30, 224, 224, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>

## Build C3D Video Classification Model

Resources
- https://towardsdatascience.com/step-by-step-implementation-3d-convolutional-neural-network-in-keras-12efbdd7b130
- https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7410867&tag=1

In [12]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
# from keras.utils import to_categorical

In [14]:
#creating C3D Model

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    inputs = keras.Input(shape=(30, 224, 224, 3)) #full shape w/ batch dim = (None, 30, 224, 224, 3) 

    x = Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1))(inputs)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1,1,1))(x)

    x = Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

    x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

    x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

    x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

    x = Dense(4096, activation="relu")(x)
    output = Dense(4096, activation="softmax")(x)

    c3d_model = keras.Model(inputs, output)

    c3d_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = c3d_model.fit(train_videos, 
                        train_labels,
                        validation_split = 0.2,
                        epochs = 16,
                        verbose= 1)

print('Done training.')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2022-07-25 02:55:48.542697: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_7452"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:68"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUC

Epoch 1/16
INFO:tensorflow:batch_all_reduce: 20 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 20 all-reduces with algorithm = nccl, num_packs = 1


2022-07-25 02:56:08.219523: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.61GiB (rounded to 2800738304)requested by op model_1/max_pooling3d_5/MaxPool3D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-25 02:56:08.219640: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-07-25 02:56:08.219696: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 74, Chunks in use: 72. 18.5KiB allocated for chunks. 18.0KiB in use in bin. 2.4KiB client-requested in use in bin.
2022-07-25 02:56:08.219713: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 7, Chunks in use: 7. 3.5KiB allocated for chunks. 3.5KiB in use in bin. 3.3KiB client-requested in use in bin.
2022-07-2

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/max_pooling3d_5/MaxPool3D' defined at (most recent call last):
    File "/usr/lib/python3.8/threading.py", line 890, in _bootstrap
      self._bootstrap_inner()
    File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
      self.run()
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1000, in run_step
      outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 859, in train_step
      y_pred = self(x, training=True)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
      return fn(*args, **kwargs)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/base_layer.py", line 1096, in __call__
      outputs = call_fn(inputs, *args, **kwargs)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 92, in error_handler